# 개선해야할 점

    1) 한자제거
    2) [사진,포토] 등 불필요한 단어 제거  OK
    3) 단어분포도(word distribution)
    4) 불용어처리 가능하게 하기 
    5) 텍스트유사도 분석                  OK
    6) 유사도 60%넘는거 삭제              OK
    7) 0점처리-건우오빠한테 물어보기
    

# 1.데이터 로드 (Data Load)

In [1]:
import re
import pandas as pd
import numpy as np
import json
#from bs4 import Beautiful Soup
# from nltk.corpus import Beautiful Soup
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer
import konlp
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
#헤더추가 (title,content,time)
train =pd.read_csv('dataset3000.csv',names=['title','content','time','nan'])
train.head()
train.shape
train.dtypes


title       object
content     object
time        object
nan        float64
dtype: object

In [3]:
# nan 변수버리기
train.drop(['nan'], axis='columns', inplace=True)

In [4]:
train.head()

title  \
0                 '[사진]조여정,'우아한 플라워원피스 입고''   
1              '[프로필]국세청 과장급 전보자(7월 22일 자)'   
2           '[조이HD]춘사영화제 참석한 봉준호 감독-한진원 작가'   
3                 '[사진]클라라,'시선 올킬 블루 롱드레스''   
4  '[사진]7개부문 노미네이트 기생충 봉준호 감독,'엄청난 환호 받으며''   

                                             content              time  
0  '제24회 춘사영화제는 ‘아시아를 대표하는 영화제로의 도약’을 표방한 제23회에 이...  2019-07-20 16:40  
1  '조사기획1계장 ▲경주세무서장 ▲국세청 인도네시아 주재관 ▲외교부 파견 ▲서울지방국...  2019-07-20 16:40  
2  '말레이시아 Finas(말레시아 영화진흥위)의 신임 대표인 Ahmad Idham을 ...  2019-07-20 16:40  
3  '제24회 춘사영화제는 ‘아시아를 대표하는 영화제로의 도약’을 표방한 제23회에 이...  2019-07-20 16:40  
4  '제24회 춘사영화제는 ‘아시아를 대표하는 영화제로의 도약’을 표방한 제23회에 이...  2019-07-20 16:40

## 1) [   ] 안에 있는 단어 제거( [  ] Remove)

In [5]:
train["title"] = train["title"].str.replace("\[(.*?)\]", "") # [ ] 안에 잇는 단어 없애기

In [6]:
# https://regexr.com/  #정규표현식 테스트 사이트 

In [7]:
train["title"]

0                                   '조여정,'우아한 플라워원피스 입고''
1                                 '국세청 과장급 전보자(7월 22일 자)'
2                               '춘사영화제 참석한 봉준호 감독-한진원 작가'
3                                   '클라라,'시선 올킬 블루 롱드레스''
4                    '7개부문 노미네이트 기생충 봉준호 감독,'엄청난 환호 받으며''
5                                          '함소원,'우아한 입장''
6                                         '엄태구, 지적인 남자야~'
7                                          '진기주, 매력적인 미모'
8                                     ''기생충' 이정은, 엄마 미소~'
9                                     '공명, 춘사영화제 빛내는 미소~'
10                               ''춘사영화제' 조여정, 칸이 반한 미소~'
11                                   ''춘사영화제' 문지인, 민트여신~'
12                                           '공명,'훈남 매력''
13                     '무역위, 전기프라이팬 특허권 침해 불공정무역행위 조사 개시'
14                                  '베이비페이스 이성경,'화려한 변신''
15                                           '클라라,'섹시하게''
16                           '상체 노출 클라라,'이보다 더 아찔할 수 없어''
17            

# only title

In [8]:
title=list(train['title'])

In [9]:
title

["'조여정,'우아한 플라워원피스 입고''",
 "'국세청 과장급 전보자(7월 22일 자)'",
 "'춘사영화제 참석한 봉준호 감독-한진원 작가'",
 "'클라라,'시선 올킬 블루 롱드레스''",
 "'7개부문 노미네이트 기생충 봉준호 감독,'엄청난 환호 받으며''",
 "'함소원,'우아한 입장''",
 "'엄태구, 지적인 남자야~'",
 "'진기주, 매력적인 미모'",
 "''기생충' 이정은, 엄마 미소~'",
 "'공명, 춘사영화제 빛내는 미소~'",
 "''춘사영화제' 조여정, 칸이 반한 미소~'",
 "''춘사영화제' 문지인, 민트여신~'",
 "'공명,'훈남 매력''",
 "'무역위, 전기프라이팬 특허권 침해 불공정무역행위 조사 개시'",
 "'베이비페이스 이성경,'화려한 변신''",
 "'클라라,'섹시하게''",
 "'상체 노출 클라라,'이보다 더 아찔할 수 없어''",
 "'조여정, 춘사영화제 여우주연상에 빛나는 자태'",
 "'함소원, 연하 남편을 향한 마음'",
 "'춘사영화제 참석한 기생충 봉준호 감독'",
 "'이정은,'영화 팬 여러분 사랑해요''",
 "'문지인,'섹시 레드립''",
 "'하지영,'어느덧 여배우''",
 "'김강현,'춘사영화제 왔어요''",
 "'공명,'달콤한 하트''",
 "'전여빈, 우아한 블랙 드레스'",
 "'전여빈, 춘사영화제가 주목하는 신예'",
 "'공명,'젠틀하게''",
 "'문지인, 10년차 베테랑 여배우~'",
 "'전여빈,'아름다운 미소''",
 "''춘사영화제' 이성경, 금빛 몸매'",
 "'클라라, 보기만 해도 시원한 블루 드레스'",
 "'정애연,'단아한 매력''",
 "'정애연,'춘사영화제 레드카펫 밟으며''",
 "'주지훈,'그 만의 시그널''",
 "'주지훈,'럭셔리하게''",
 "'주지훈,'오늘도 멋짐 폭발''",
 "'기생충 박명훈,'춘사영화제 수상 기대해요''",
 "'한진원 작가-봉준호 감독,'기생충 초대 받았어요''",
 "'골드 드레스 이성경,'귀걸이도 금빛 하트~''",
 "'레드

## 3)한글이 아닌 단어 지우기 (Remove symbols)

In [10]:
#전체제목
# 리스트전체가 아니라 하나씩 빼서 처리해야하기 때문에 for문으로 처리.
#1.한글 및 공백을 제외한 문자를 모두 제거
for i in range(len(title)) :
    title[i] = re.sub("[^가-힣ㄱ-ㅎ ㅏ-ㅣ\\s]","", title[i])

print(title)

['조여정우아한 플라워원피스 입고', '국세청 과장급 전보자월 일 자', '춘사영화제 참석한 봉준호 감독한진원 작가', '클라라시선 올킬 블루 롱드레스', '개부문 노미네이트 기생충 봉준호 감독엄청난 환호 받으며', '함소원우아한 입장', '엄태구 지적인 남자야', '진기주 매력적인 미모', '기생충 이정은 엄마 미소', '공명 춘사영화제 빛내는 미소', '춘사영화제 조여정 칸이 반한 미소', '춘사영화제 문지인 민트여신', '공명훈남 매력', '무역위 전기프라이팬 특허권 침해 불공정무역행위 조사 개시', '베이비페이스 이성경화려한 변신', '클라라섹시하게', '상체 노출 클라라이보다 더 아찔할 수 없어', '조여정 춘사영화제 여우주연상에 빛나는 자태', '함소원 연하 남편을 향한 마음', '춘사영화제 참석한 기생충 봉준호 감독', '이정은영화 팬 여러분 사랑해요', '문지인섹시 레드립', '하지영어느덧 여배우', '김강현춘사영화제 왔어요', '공명달콤한 하트', '전여빈 우아한 블랙 드레스', '전여빈 춘사영화제가 주목하는 신예', '공명젠틀하게', '문지인 년차 베테랑 여배우', '전여빈아름다운 미소', '춘사영화제 이성경 금빛 몸매', '클라라 보기만 해도 시원한 블루 드레스', '정애연단아한 매력', '정애연춘사영화제 레드카펫 밟으며', '주지훈그 만의 시그널', '주지훈럭셔리하게', '주지훈오늘도 멋짐 폭발', '기생충 박명훈춘사영화제 수상 기대해요', '한진원 작가봉준호 감독기생충 초대 받았어요', '골드 드레스 이성경귀걸이도 금빛 하트', '레드카펫 런웨이 이성경모델은 달라도 달라', '이성경금빛 각선미', '조여정미모 여전해', '조여정아기 같은 미소', '조여정미소 여신', '꽃보다 더 빛나는 조여정의 미모', '미소가 아름다운 조여정', '중기 해외진출 지원 발벗고 나선 부산경진원창조혁신센터', '명불허전 클라라명품 콜라병 몸매 시선집중', '공사 첨단 환경기술 눈도장', '경영 대표팀 메인풀서 첫 훈련 메달 노리는 김서영 컨디션 이상 무',

# 2.데이터 전처리 (Data Preprocessing)

# 3.텍스트 유사도 (Text similarity)

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## 명사만 추가한 리스트 만들기 (extraction only nouns)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt

okt=Okt()

doc_nouns_list=[]

for doc in title:
    nouns=okt.nouns(doc)
    doc_nouns=''
    
    for noun in nouns:
        doc_nouns+=noun + ' '
    doc_nouns_list.append(doc_nouns)
    
for i in range(0,len(title)):
    print('doc' +str(i+1)+ ' : ' +str(doc_nouns_list[i]))

C:\Users\glass\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


doc1 : 조여정 플라워 원피스 
doc2 : 국세청 과장 전보 자월 일 자 
doc3 : 영화제 참석 봉준호 감독 진원 작가 
doc4 : 클라라 시선 올킬 블루 롱 드레스 
doc5 : 개 부문 노미네이트 기생충 봉준호 감독 환호 
doc6 : 함소원 입장 
doc7 : 엄태구 지적 남자 
doc8 : 진기 주 매력 미모 
doc9 : 기생충 이정은 엄마 미소 
doc10 : 공명 영화제 빛 미소 
doc11 : 영화제 조여정 칸 반한 미소 
doc12 : 영화제 문지 민트 여신 
doc13 : 공명 훈남 매력 
doc14 : 무역 위 전기 프라이팬 특허권 침해 불공정 무역 행위 조사 개시 
doc15 : 베이비페이스 이성경 변신 
doc16 : 클라라 
doc17 : 상체 노출 클라라 더 수 
doc18 : 조여정 영화제 여우주연상 자태 
doc19 : 함소원 연하 남편 향 마음 
doc20 : 영화제 참석 기생충 봉준호 감독 
doc21 : 이정은 영화 팬 여러분 사랑 
doc22 : 지인 섹시 레드 립 
doc23 : 하지영 여배우 
doc24 : 김강현 영화제 
doc25 : 공명 하트 
doc26 : 빈 블랙 드레스 
doc27 : 빈 영화제 주목 신예 
doc28 : 공명 젠틀 
doc29 : 문지 년차 베테 여배우 
doc30 : 빈 미소 
doc31 : 영화제 이성경 금빛 몸매 
doc32 : 클라라 보기 해도 블루 드레스 
doc33 : 정애연 단아 매력 
doc34 : 정애연 영화제 레드카펫 
doc35 : 주지훈 그 시그널 
doc36 : 주지훈 럭셔리 
doc37 : 주지훈 오늘 폭발 
doc38 : 기생충 박명 훈춘 영화제 수상 
doc39 : 진원 작가 봉준호 감독 기생충 초대 
doc40 : 골드 드레스 이성경 귀걸이 금빛 하트 
doc41 : 레드카펫 런웨이 이성경 모델 달라 달라 
doc42 : 이성경 금빛 각선미 
doc43 : 조여정 미모 
doc44 : 조여정 아기 미소 
doc45 : 조여정 미소 여신 
doc46 :

doc1755 : 국내 최대 스타트업 페어 넥스트 라이즈 서울 개막 한국판 
doc1756 : 처 스타트업 투자 유치 개막 
doc1757 : 산 제의 처붐 확산 넥스트 라이즈 서울 개막 
doc1758 : 처 대기업 투자자 하나로 넥스트 라이즈 개최 
doc1759 : 후추 커피 유행 예측 
doc1760 : 글로벌 스타트업 페어 넥스트 라이즈 개막 이동걸 혁신 성장 플랫폼 
doc1761 : 산업 은행 성황리 개최 이동걸 
doc1762 : 준 먼저 부양책 가능성 
doc1763 : 산업 은행 무역 협회 등 혁신 성장 개최 
doc1764 : 글로벌 기업 중 신규 진입 개 
doc1765 : 한국 부평 공장 년 뒤 문 
doc1766 : 세계 경제 미국 나 홀로 성장 이유 
doc1767 : 외환시세 월 일 하나은행 차 고시 기준 
doc1768 : 유니콘 탄생 인도네시아 핫 이유 
doc1769 : 숀 데스 팔뚝 타투 열정 무대 
doc1770 : 제시카 농염 포즈 자태 수영복 몸매 과시 
doc1771 : 필리핀 의 새 거점 외국인 명 자폭 테러 교육 
doc1772 : 조선대 치과 병원 컬 코리아 아카데미 글로벌 연수 진행 
doc1773 : 일 인도네시아 루피아 오늘 환율 감소 
doc1774 : 월 일 인도네시아 루피아 오늘 환율 정보 
doc1775 : 발리섬 마약 소지 체포 호주인 죄수복 차림 거리 행진 망신 
doc1776 : 랜드 인도네시아 출격 확정 공식 
doc1777 : 랜드 출격 확정 공식 
doc1778 : 필리핀 의 새 거점 외국인 명 자폭 테러 교육 
doc1779 : 크래프톤 미스트 오버 아시아 지역 퍼블리싱 계약 체결 
doc1780 : 글로벌 루키 걸그룹 랜드 출격 확정 
doc1781 : 크래프톤 아크 시스템 웍스 미스트 오버 아시아 서비스 계약 
doc1782 : 크래프톤 아시아 미스트 오버 
doc1783 : 크래프톤 미스트 오버 아시아 퍼블리싱 계약 체결 
doc1784 : 랜드 출격 확정 글로벌 위상 고공 행진 


In [13]:
tfidf_vectorizer=TfidfVectorizer(min_df=1)
tfidf_matrix = tfidf_vectorizer.fit_transform(doc_nouns_list)

document_distances=(tfidf_matrix*tfidf_matrix.T)

print('유사도 분석을 위한' + str(document_distances.get_shape()[0])+'x'+str(document_distances.get_shape()[1])+'matrix를 만들었습니다')
print(document_distances.toarray())    

유사도 분석을 위한2873x2873matrix를 만들었습니다
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


# 중복줄이기 (Reduce duplication Sentence)


In [14]:
distanceArray = document_distances.toarray()
check = [True for _ in range(len(distanceArray))]

In [15]:
for i in range(len(distanceArray)):
    if(check[i] == False):
        continue
    for j in range(len(distanceArray)):
        if(i == j):
            continue
        else :
            if(check[j] == True and distanceArray[i][j] >= 0.6):
                check[j] = False

In [16]:
filterList = []
print(len(check))
for i in range(len(check)):
    if(check[i] == True):
        filterList.append(title[i])
print(len(filterList))

2873
1675


In [17]:
print(filterList)

['조여정우아한 플라워원피스 입고', '국세청 과장급 전보자월 일 자', '춘사영화제 참석한 봉준호 감독한진원 작가', '클라라시선 올킬 블루 롱드레스', '개부문 노미네이트 기생충 봉준호 감독엄청난 환호 받으며', '함소원우아한 입장', '엄태구 지적인 남자야', '진기주 매력적인 미모', '기생충 이정은 엄마 미소', '공명 춘사영화제 빛내는 미소', '춘사영화제 조여정 칸이 반한 미소', '춘사영화제 문지인 민트여신', '공명훈남 매력', '무역위 전기프라이팬 특허권 침해 불공정무역행위 조사 개시', '베이비페이스 이성경화려한 변신', '클라라섹시하게', '상체 노출 클라라이보다 더 아찔할 수 없어', '조여정 춘사영화제 여우주연상에 빛나는 자태', '함소원 연하 남편을 향한 마음', '이정은영화 팬 여러분 사랑해요', '문지인섹시 레드립', '하지영어느덧 여배우', '김강현춘사영화제 왔어요', '공명달콤한 하트', '전여빈 우아한 블랙 드레스', '전여빈 춘사영화제가 주목하는 신예', '공명젠틀하게', '문지인 년차 베테랑 여배우', '전여빈아름다운 미소', '춘사영화제 이성경 금빛 몸매', '클라라 보기만 해도 시원한 블루 드레스', '정애연단아한 매력', '정애연춘사영화제 레드카펫 밟으며', '주지훈그 만의 시그널', '주지훈럭셔리하게', '주지훈오늘도 멋짐 폭발', '기생충 박명훈춘사영화제 수상 기대해요', '골드 드레스 이성경귀걸이도 금빛 하트', '레드카펫 런웨이 이성경모델은 달라도 달라', '조여정미모 여전해', '조여정아기 같은 미소', '중기 해외진출 지원 발벗고 나선 부산경진원창조혁신센터', '명불허전 클라라명품 콜라병 몸매 시선집중', '공사 첨단 환경기술 눈도장', '경영 대표팀 메인풀서 첫 훈련 메달 노리는 김서영 컨디션 이상 무', '김주희 무용가 두리안 먹방 맛있는데 다채로운 매력캄보디아 여행 ', '경북 중소기업 수출 지원 러시아인니에 교두보', '이철우 경북도지사 변해야 산다 도정의 국제경쟁력 강화에 새바람', '  세계테마

# 4. 형태소 나누기 (Stemming Nouns)

In [18]:
from konlpy.tag import Okt

In [19]:

okt2=Okt() 
title_text2=[ ] # or title_text=[''] * len(title)
for i in range(len(filterList)) :
    title_text2.append(okt2.nouns(title[i]))

print(title_text2[:10])

[['조여정', '플라워', '원피스'], ['국세청', '과장', '전보', '자월', '일', '자'], ['영화제', '참석', '봉준호', '감독', '진원', '작가'], ['클라라', '시선', '올킬', '블루', '롱', '드레스'], ['개', '부문', '노미네이트', '기생충', '봉준호', '감독', '환호'], ['함소원', '입장'], ['엄태구', '지적', '남자'], ['진기', '주', '매력', '미모'], ['기생충', '이정은', '엄마', '미소'], ['공명', '영화제', '빛', '미소']]


In [20]:
doc_nouns_list

['조여정 플라워 원피스 ',
 '국세청 과장 전보 자월 일 자 ',
 '영화제 참석 봉준호 감독 진원 작가 ',
 '클라라 시선 올킬 블루 롱 드레스 ',
 '개 부문 노미네이트 기생충 봉준호 감독 환호 ',
 '함소원 입장 ',
 '엄태구 지적 남자 ',
 '진기 주 매력 미모 ',
 '기생충 이정은 엄마 미소 ',
 '공명 영화제 빛 미소 ',
 '영화제 조여정 칸 반한 미소 ',
 '영화제 문지 민트 여신 ',
 '공명 훈남 매력 ',
 '무역 위 전기 프라이팬 특허권 침해 불공정 무역 행위 조사 개시 ',
 '베이비페이스 이성경 변신 ',
 '클라라 ',
 '상체 노출 클라라 더 수 ',
 '조여정 영화제 여우주연상 자태 ',
 '함소원 연하 남편 향 마음 ',
 '영화제 참석 기생충 봉준호 감독 ',
 '이정은 영화 팬 여러분 사랑 ',
 '지인 섹시 레드 립 ',
 '하지영 여배우 ',
 '김강현 영화제 ',
 '공명 하트 ',
 '빈 블랙 드레스 ',
 '빈 영화제 주목 신예 ',
 '공명 젠틀 ',
 '문지 년차 베테 여배우 ',
 '빈 미소 ',
 '영화제 이성경 금빛 몸매 ',
 '클라라 보기 해도 블루 드레스 ',
 '정애연 단아 매력 ',
 '정애연 영화제 레드카펫 ',
 '주지훈 그 시그널 ',
 '주지훈 럭셔리 ',
 '주지훈 오늘 폭발 ',
 '기생충 박명 훈춘 영화제 수상 ',
 '진원 작가 봉준호 감독 기생충 초대 ',
 '골드 드레스 이성경 귀걸이 금빛 하트 ',
 '레드카펫 런웨이 이성경 모델 달라 달라 ',
 '이성경 금빛 각선미 ',
 '조여정 미모 ',
 '조여정 아기 미소 ',
 '조여정 미소 여신 ',
 '꽃 더 조여정 미모 ',
 '미소 조여정 ',
 '중기 해외진출 지원 발 나선 부산 경진 창조 혁신 센터 ',
 '명불허전 클라라 명품 콜라 병 몸매 시선집중 ',
 '공사 첨단 환경기술 눈 도장 ',
 '경영 대표팀 메인 풀 첫 훈련 메달 김서영 컨디션 이상 무 ',
 '김주희 무용가 두리안 먹방 채 매력 캄보디

##  5) 불용어 처리 ( Stop words remove)

In [21]:
def tokenize_new():
    x = [line.rstrip('\n') for line in open('C:/Users/glass\Desktop/AI공모전_4-2/pratice/stop_word.txt', 'r', encoding='utf-8')] #리스트
    word_tokens = title_text2
    result = [] 
    
    for w in word_tokens: # w; 리스트하나 (조여정,플라워,원피스)
        temp = []
        for noun in w :  # nouns: 이중리스트의 원소 (조여정/ 플라워 /원피스)
            if noun not in x: # x에 noun이 없으면 temp에 넣어라.
                temp.append(noun) # temp(리스트) ->  result에 저장 -> 위로 올라가서 temp를 초기화시키고 다음줄을 확인.. 
        result.append(temp)
    return result

In [22]:
tokenize_new()

[['조여정', '플라워', '원피스'],
 ['국세청', '과장', '전보', '자월'],
 ['영화제', '참석', '봉준호', '감독', '진원', '작가'],
 ['클라라', '시선', '올킬', '블루', '드레스'],
 ['개', '부문', '노미네이트', '기생충', '봉준호', '감독', '환호'],
 ['함소원', '입장'],
 ['엄태구', '지적', '남자'],
 ['진기', '매력', '미모'],
 ['기생충', '이정은', '엄마', '미소'],
 ['공명', '영화제', '빛', '미소'],
 ['영화제', '조여정', '칸', '반한', '미소'],
 ['영화제', '문지', '민트', '여신'],
 ['공명', '훈남', '매력'],
 ['무역', '위', '전기', '프라이팬', '특허권', '침해', '불공정', '무역', '행위', '조사', '개시'],
 ['베이비페이스', '이성경', '변신'],
 ['클라라'],
 ['상체', '노출', '클라라'],
 ['조여정', '영화제', '여우주연상', '자태'],
 ['함소원', '연하', '남편', '향', '마음'],
 ['영화제', '참석', '기생충', '봉준호', '감독'],
 ['이정은', '영화', '팬', '여러분', '사랑'],
 ['지인', '섹시', '레드', '립'],
 ['하지영', '여배우'],
 ['김강현', '영화제'],
 ['공명', '하트'],
 ['빈', '블랙', '드레스'],
 ['빈', '영화제', '주목', '신예'],
 ['공명', '젠틀'],
 ['문지', '년차', '베테', '여배우'],
 ['빈', '미소'],
 ['영화제', '이성경', '금빛', '몸매'],
 ['클라라', '보기', '해도', '블루', '드레스'],
 ['정애연', '단아', '매력'],
 ['정애연', '영화제', '레드카펫'],
 ['주지훈', '시그널'],
 ['주지훈', '럭셔리'],
 ['주지훈', '오늘', '폭발'],
 ['기생충', '박명'

In [23]:
title_text=tokenize_new()

# 5.word2vec 벡터화

In [24]:
#진행상황확인위해
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)



In [25]:
from gensim.models import Word2Vec
print("Training model...")
model= Word2Vec(title_text, size=300, window=5, min_count=5, workers=4, sg=1)

2019-12-03 16:25:26,270 : INFO : collecting all words and their counts
2019-12-03 16:25:26,271 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-12-03 16:25:26,280 : INFO : collected 2752 word types from a corpus of 11007 raw words and 1675 sentences
2019-12-03 16:25:26,281 : INFO : Loading a fresh vocabulary
2019-12-03 16:25:26,286 : INFO : effective_min_count=5 retains 565 unique words (20% of original 2752, drops 2187)
2019-12-03 16:25:26,287 : INFO : effective_min_count=5 leaves 7388 word corpus (67% of original 11007, drops 3619)
2019-12-03 16:25:26,291 : INFO : deleting the raw counts dictionary of 2752 items
2019-12-03 16:25:26,292 : INFO : sample=0.001 downsamples 81 most-common words
2019-12-03 16:25:26,293 : INFO : downsampling leaves estimated 6105 word corpus (82.6% of prior 7388)
2019-12-03 16:25:26,296 : INFO : estimated required memory for 565 words and 300 dimensions: 1638500 bytes
2019-12-03 16:25:26,297 : INFO : resetting layer weights


Training model...


2019-12-03 16:25:26,534 : INFO : training model with 4 workers on 565 vocabulary and 300 features, using sg=1 hs=0 sample=0.001 negative=5 window=5
2019-12-03 16:25:26,546 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-12-03 16:25:26,548 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-12-03 16:25:26,551 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-12-03 16:25:26,574 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-12-03 16:25:26,575 : INFO : EPOCH - 1 : training on 11007 raw words (6073 effective words) took 0.0s, 181602 effective words/s
2019-12-03 16:25:26,585 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-12-03 16:25:26,587 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-12-03 16:25:26,590 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-12-03 16:25:26,616 : INFO : worker thread finished; awaiting finish of 0 more 

In [26]:
# 모델저장
model_name="300features_5minwords_5context"
model.save(model_name)

2019-12-03 16:25:26,770 : INFO : saving Word2Vec object under 300features_5minwords_5context, separately None
2019-12-03 16:25:26,771 : INFO : not storing attribute vectors_norm
2019-12-03 16:25:26,773 : INFO : not storing attribute cum_table
2019-12-03 16:25:26,791 : INFO : saved 300features_5minwords_5context


In [27]:
model.most_similar("글로벌", topn=10)

C:\Users\glass\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
2019-12-03 16:25:26,800 : INFO : precomputing L2-norms of word weight vectors


[('한국', 0.9996827840805054),
 ('인도네시아', 0.9996762275695801),
 ('인니', 0.9995836019515991),
 ('개최', 0.99956214427948),
 ('세계', 0.9995616674423218),
 ('인증', 0.9995607733726501),
 ('스타', 0.999550461769104),
 ('시장', 0.9995142221450806),
 ('위', 0.9995078444480896),
 ('기업', 0.9995073080062866)]